![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/nlu/blob/master/examples/colab/Training/binary_text_classification/NLU_training_sentiment_classifier_demo_finanical_news.ipynb)



# Training a Sentiment Analysis Classifier with NLU 
## 2 class Finance News sentiment classifier training
With the [SentimentDL model](https://nlp.johnsnowlabs.com/docs/en/annotators#sentimentdl-multi-class-sentiment-analysis-annotator) from Spark NLP you can achieve State Of the Art results on any multi class text classification problem 

This notebook showcases the following features : 

- How to train the deep learning classifier
- How to store a pipeline to disk
- How to load the pipeline from disk (Enables NLU offline mode)



# 1. Install Java 8 and NLU

In [ ]:
import os
from sklearn.metrics import classification_report
! apt-get update -qq > /dev/null   
# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! pip install nlu pyspark==2.4.7 > /dev/null  

import nlu

# 2. Download Finanical News  Sentiment dataset 
https://www.kaggle.com/ankurzing/sentiment-analysis-for-financial-news

This dataset contains the sentiments for financial news headlines from the perspective of a retail investor. Further details about the dataset can be found in: Malo, P., Sinha, A., Takala, P., Korhonen, P. and Wallenius, J. (2014): “Good debt or bad debt: Detecting semantic orientations in economic texts.” Journal of the American Society for Information Science and Technology.

In [ ]:
! wget http://ckl-it.de/wp-content/uploads/2021/01/all-data.csv


--2021-01-16 09:06:20--  http://ckl-it.de/wp-content/uploads/2021/01/all-data.csv
Resolving ckl-it.de (ckl-it.de)... 217.160.0.108, 2001:8d8:100f:f000::209
Connecting to ckl-it.de (ckl-it.de)|217.160.0.108|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 704799 (688K) [text/csv]
Saving to: ‘all-data.csv’

all-data.csv        100%[===================>] 688.28K  1.09MB/s    in 0.6s    

2021-01-16 09:06:21 (1.09 MB/s) - ‘all-data.csv’ saved [704799/704799]



In [ ]:
import pandas as pd
train_path = '/content/all-data.csv'

train_df = pd.read_csv(train_path)
# the text data to use for classification should be in a column named 'text'
# the label column must have name 'y' name be of type str
columns=['text','y']
train_df = train_df[columns]
train_df = train_df[~train_df["y"].isin(["neutral"])]
train_df

,text,y
1,The international electronic industry company ...,negative
2,With the new production plant the company woul...,positive
3,According to the company 's updated strategy f...,positive
4,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,positive
5,"For the last quarter of 2010 , Componenta 's n...",positive
...,...,...
4839,HELSINKI Thomson Financial - Shares in Cargote...,negative
4840,LONDON MarketWatch -- Share prices ended lower...,negative
4842,Operating profit fell to EUR 35.4 mn from EUR ...,negative
4843,Net sales of the Paper segment decreased to EU...,negative


# 3. Train Deep Learning Classifier using nlu.load('train.sentiment')

You dataset label column should be named 'y' and the feature column with text data should be named 'text'

In [ ]:
import nlu 
# load a trainable pipeline by specifying the train. prefix  and fit it on a datset with label and text columns
# by default the Universal Sentence Encoder (USE) Sentence embeddings are used for generation
trainable_pipe = nlu.load('train.sentiment')
fitted_pipe = trainable_pipe.fit(train_df.iloc[:50])

# predict with the trainable pipeline on dataset and get predictions
preds = fitted_pipe.predict(train_df.iloc[:50],output_level='document')
#sentence detector that is part of the pipe generates sone NaNs. lets drop them first
preds.dropna(inplace=True)
print(classification_report(preds['y'], preds['sentiment']))

preds

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         1
    positive       0.98      1.00      0.99        49

    accuracy                           0.98        50
   macro avg       0.49      0.50      0.49        50
weighted avg       0.96      0.98      0.97        50



,document,sentiment,y,sentiment_confidence,text,default_name_embeddings
origin_index,,,,,,
1,The international electronic industry company ...,positive,negative,1.000000,The international electronic industry company ...,"[0.002136496128514409, 0.07194118946790695, -0..."
2,With the new production plant the company woul...,positive,positive,1.000000,With the new production plant the company woul...,"[0.05198746547102928, 0.03577739745378494, -0...."
3,According to the company 's updated strategy f...,positive,positive,1.000000,According to the company 's updated strategy f...,"[0.03416536748409271, 0.04053246229887009, -0...."
4,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,positive,positive,1.000000,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,"[0.07730763405561447, -0.045694783329963684, -..."
5,"For the last quarter of 2010 , Componenta 's n...",positive,positive,1.000000,"For the last quarter of 2010 , Componenta 's n...","[0.05603468790650368, 0.04817350581288338, -0...."
6,"In the third quarter of 2010 , net sales incre...",positive,positive,1.000000,"In the third quarter of 2010 , net sales incre...","[0.037710510194301605, 0.037198420614004135, -..."
7,Operating profit rose to EUR 13.1 mn from EUR ...,positive,positive,1.000000,Operating profit rose to EUR 13.1 mn from EUR ...,"[0.04557091370224953, 0.0453636609017849, -0.0..."
8,"Operating profit totalled EUR 21.1 mn , up fro...",positive,positive,1.000000,"Operating profit totalled EUR 21.1 mn , up fro...","[0.05191247910261154, 0.059505216777324677, -0..."
9,TeliaSonera TLSN said the offer is in line wit...,positive,positive,1.000000,TeliaSonera TLSN said the offer is in line wit...,"[0.07441692799329758, -0.0487477071583271, -0...."


# Test the fitted pipe on new example

In [ ]:
fitted_pipe.predict('According to the most recent update there has been a major decrese in the rate of oil')

,document,sentiment,sentiment_confidence,default_name_embeddings
origin_index,,,,
0,Bitcoin is going to the moon!,positive,0.999994,"[0.06468033790588379, -0.040837567299604416, -..."


## Configure pipe training parameters

In [ ]:
trainable_pipe.print_info()

The following parameters are configurable for this NLU pipeline (You can copy paste the examples) :
>>> pipe['sentiment_dl'] has settable params:
pipe['sentiment_dl'].setMaxEpochs(2)                 | Info: Maximum number of epochs to train | Currently set to : 2
pipe['sentiment_dl'].setLr(0.005)                    | Info: Learning Rate | Currently set to : 0.005
pipe['sentiment_dl'].setBatchSize(64)                | Info: Batch size | Currently set to : 64
pipe['sentiment_dl'].setDropout(0.5)                 | Info: Dropout coefficient | Currently set to : 0.5
pipe['sentiment_dl'].setEnableOutputLogs(True)       | Info: Whether to use stdout in addition to Spark logs. | Currently set to : True
pipe['sentiment_dl'].setThreshold(0.6)               | Info: The minimum threshold for the final result otheriwse it will be neutral | Currently set to : 0.6
pipe['sentiment_dl'].setThresholdLabel('neutral')    | Info: In case the score is less than threshold, what should be the label. Default i

## Retrain with new parameters

In [ ]:
# Train longer!
trainable_pipe['sentiment_dl'].setMaxEpochs(5)  
fitted_pipe = trainable_pipe.fit(train_df.iloc[:100])
# predict with the trainable pipeline on dataset and get predictions
preds = fitted_pipe.predict(train_df.iloc[:100],output_level='document')

#sentence detector that is part of the pipe generates sone NaNs. lets drop them first
preds.dropna(inplace=True)
print(classification_report(preds['y'], preds['sentiment']))

preds

              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         1
    positive       0.99      1.00      0.99        99

    accuracy                           0.99       100
   macro avg       0.49      0.50      0.50       100
weighted avg       0.98      0.99      0.99       100



,document,sentiment,y,sentiment_confidence,text,default_name_embeddings
origin_index,,,,,,
1,The international electronic industry company ...,positive,negative,1.000000,The international electronic industry company ...,"[0.002136496128514409, 0.07194118946790695, -0..."
2,With the new production plant the company woul...,positive,positive,1.000000,With the new production plant the company woul...,"[0.05198746547102928, 0.03577739745378494, -0...."
3,According to the company 's updated strategy f...,positive,positive,1.000000,According to the company 's updated strategy f...,"[0.03416536748409271, 0.04053246229887009, -0...."
4,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,positive,positive,1.000000,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,"[0.07730763405561447, -0.045694783329963684, -..."
5,"For the last quarter of 2010 , Componenta 's n...",positive,positive,1.000000,"For the last quarter of 2010 , Componenta 's n...","[0.05603468790650368, 0.04817350581288338, -0...."
...,...,...,...,...,...,...
116,Operating profit margin increased from 11.2 % ...,positive,positive,1.000000,Operating profit margin increased from 11.2 % ...,"[0.01058729737997055, -0.008798183873295784, -..."
117,Operating profit rose to EUR 3.11 mn from EUR ...,positive,positive,1.000000,Operating profit rose to EUR 3.11 mn from EUR ...,"[0.03610285371541977, 0.04256380349397659, -0...."
118,Operating profit rose to EUR 5mn from EUR 2.8 ...,positive,positive,1.000000,Operating profit rose to EUR 5mn from EUR 2.8 ...,"[0.04815328121185303, 0.050376053899526596, -0..."


# Try training with different Embeddings

In [ ]:
# We can use nlu.print_components(action='embed_sentence') to see every possibler sentence embedding we could use. Lets use bert!
nlu.print_components(action='embed_sentence')

For language <en> NLU provides the following Models : 
nlu.load('en.embed_sentence') returns Spark NLP model tfhub_use
nlu.load('en.embed_sentence.use') returns Spark NLP model tfhub_use
nlu.load('en.embed_sentence.tfhub_use') returns Spark NLP model tfhub_use
nlu.load('en.embed_sentence.use.lg') returns Spark NLP model tfhub_use_lg
nlu.load('en.embed_sentence.tfhub_use.lg') returns Spark NLP model tfhub_use_lg
nlu.load('en.embed_sentence.albert') returns Spark NLP model albert_base_uncased
nlu.load('en.embed_sentence.electra') returns Spark NLP model sent_electra_small_uncased
nlu.load('en.embed_sentence.electra_small_uncased') returns Spark NLP model sent_electra_small_uncased
nlu.load('en.embed_sentence.electra_base_uncased') returns Spark NLP model sent_electra_base_uncased
nlu.load('en.embed_sentence.electra_large_uncased') returns Spark NLP model sent_electra_large_uncased
nlu.load('en.embed_sentence.bert') returns Spark NLP model sent_bert_base_uncased
nlu.load('en.embed_sentenc

In [ ]:
trainable_pipe = nlu.load('en.embed_sentence.small_bert_L12_768 train.sentiment')
# We need to train longer and user smaller LR for NON-USE based sentence embeddings usually
# We could tune the hyperparameters further with hyperparameter tuning methods like gridsearch
# Also longer training gives more accuracy
trainable_pipe['sentiment_dl'].setMaxEpochs(70)  
trainable_pipe['sentiment_dl'].setLr(0.0005) 
fitted_pipe = trainable_pipe.fit(train_df)
# predict with the trainable pipeline on dataset and get predictions
preds = fitted_pipe.predict(train_df,output_level='document')

#sentence detector that is part of the pipe generates sone NaNs. lets drop them first
preds.dropna(inplace=True)
print(classification_report(preds['y'], preds['sentiment']))

#preds

sent_small_bert_L12_768 download started this may take some time.
Approximate size to download 392.9 MB
[OK!]
              precision    recall  f1-score   support

    negative       0.88      0.87      0.88       604
     neutral       0.00      0.00      0.00         0
    positive       0.96      0.92      0.94      1363

    accuracy                           0.91      1967
   macro avg       0.62      0.60      0.61      1967
weighted avg       0.94      0.91      0.92      1967



# 5. Lets save the model

In [ ]:
stored_model_path = './models/classifier_dl_trained' 
fitted_pipe.save(stored_model_path)

Stored model in ./models/classifier_dl_trained


# 6. Lets load the model from HDD.
This makes Offlien NLU usage possible!   
You need to call nlu.load(path=path_to_the_pipe) to load a model/pipeline from disk.

In [ ]:
hdd_pipe = nlu.load(path=stored_model_path)

preds = hdd_pipe.predict('According to the most recent update there has been a major decrese in the rate of oil')
preds

Fitting on empty Dataframe, could not infer correct training method!


,document,sentiment,sentiment_confidence,en_embed_sentence_small_bert_L12_768_embeddings
origin_index,,,,
0,Tesla plans to invest 10M into the ML sector,positive,0.999980,"[0.15737222135066986, 0.2598555386066437, 0.85..."


In [ ]:
hdd_pipe.print_info()

The following parameters are configurable for this NLU pipeline (You can copy paste the examples) :
>>> pipe['document_assembler'] has settable params:
pipe['document_assembler'].setCleanupMode('shrink')            | Info: possible values: disabled, inplace, inplace_full, shrink, shrink_full, each, each_full, delete_full | Currently set to : shrink
>>> pipe['sentence_detector'] has settable params:
pipe['sentence_detector'].setCustomBounds([])                  | Info: characters used to explicitly mark sentence bounds | Currently set to : []
pipe['sentence_detector'].setDetectLists(True)                 | Info: whether detect lists during sentence detection | Currently set to : True
pipe['sentence_detector'].setExplodeSentences(False)           | Info: whether to explode each sentence into a different row, for better parallelization. Defaults to false. | Currently set to : False
pipe['sentence_detector'].setMaxLength(99999)                  | Info: Set the maximum allowed length for ea